In [1]:
import mysql.connector as mysql
import pandas as pd
import numpy as np


import datetime
import calendar

In [2]:
# Connect to Database
def connect_db():
    # enter your server IP address/domain name
    HOST = "xx.xxx.xxx.xxx" # or "domain.com"
    # database name, if you want just to connect to MySQL server, leave it empty
    DATABASE = "db_name"
    # this is the user you create
    USER = "username"
    # user password
    PASSWORD = "your_password"
    # connect to MySQL server
    db_connection = mysql.connect(host=HOST, database=DATABASE, user=USER, password=PASSWORD)
    print("Connected to:", db_connection.get_server_info())
    # enter your code here!

    return db_connection

def is_end_of_month(date_str):
    date_object = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    _, last_day = calendar.monthrange(date_object.year, date_object.month)
    
    return date_object.day == last_day

def is_first_of_month(date_str):
    date_object = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    
    return date_object.day == 1

def adjust_date(data):
    for i, row in data.iterrows():
        # current_date = row['adj_date']
        current_hour = row['created_ticket'].hour
        
        if (current_hour >= 22):
            new_date = row['created_ticket'].date() + datetime.timedelta(days=1)
            data.at[i, 'adj_date'] = new_date
    
    return data

In [3]:
# Collect Data
def collect_data(start, end):

    # connect database
    db_connection = connect_db()

    # create a cursor object
    cursor_query_booking_detail = db_connection.cursor()

    #
    if is_first_of_month(start):
        start = datetime.datetime.strptime(start, '%Y-%m-%d') - datetime.timedelta(hours=2)
    
    if is_end_of_month(end):
        end = datetime.datetime.strptime(end, '%Y-%m-%d') + datetime.timedelta(hours=22)

    # execute a SELECT query
    query_booking_detail = f"""
    SELECT 
        tb.created_on AS created_ticket, 
        booking_id,
        tb.status, 
        status_name,
        tb.po_id, 
        po_name, 
        route_info, 
        shelter_id, 
        shelter_name, 
        price, 
        sharing_po,
        CAST(price*(sharing_po/100) AS SIGNED) as revenue_po,
        CAST(price*(sharing_ap2/100) AS SIGNED) as revenue_ap2,
        CAST((price*(sharing_ap2/100))*0.28 AS SIGNED) as revenue_apsd,
        payment_channel_id AS payment_id, 
        ms_payment_type.payment_type AS payment_method
    FROM t_trx_booking_detail AS tb
    JOIN t_mtr_shelter ON tb.shelter_id = t_mtr_shelter.id_seq
    JOIN t_mtr_po ON tb.po_id = t_mtr_po.id_seq
    JOIN t_mtr_ticket_status ON tb.status = t_mtr_ticket_status.status_code
    JOIN ms_payment_type ON tb.payment_channel_id = ms_payment_type.id
    JOIN ms_revenue ON tb.po_id = ms_revenue.po_id
    WHERE tb.created_on < '{end}' AND tb.created_on >= '{start}'
    ORDER BY tb.created_on
    """

    # Execute query
    print(f"status: collecting data from {start} to {end}")
    cursor_query_booking_detail.execute(query_booking_detail)
    result_cursor_query_booking_detail = cursor_query_booking_detail.fetchall()
    print(f"status: success collect data")

    # Get column names from the cursor description
    column_names_booking_detail = [column[0] for column in cursor_query_booking_detail.description]

    # Convert the result_set to a DataFrame
    data = pd.DataFrame(result_cursor_query_booking_detail, columns=column_names_booking_detail)
    data['date'] = data['created_ticket'].dt.date
    data['adj_date'] = data['created_ticket'].dt.date
    
    print(f"status: adjusting date")
    data = adjust_date(data)

    data['date'] = pd.to_datetime(data['date'])
    data['adj_date'] = pd.to_datetime(data['adj_date'])

    # drop unused data
    print(f"status: drop unused data")
    df = data[(~data['status_name'].isin(['NOT PAID', 'CANCELED'])) & (data['po_name'] != 'PO Test APSD')].copy()
    
    return df

In [4]:
def get_recap_data(start, end, file_name):

    data = collect_data(start, end)

    ## RECAP PO
    print("status: starting recap po")
    jumlah_produksi = data.groupby(['po_name', 'adj_date']).size()
    jumlah_pendapatan = data.groupby(['po_name', 'adj_date'])['price'].sum()
    pendapatan_po = data.groupby(['po_name', 'adj_date'])['revenue_po'].sum()
    pendapatan_ap2 = data.groupby(['po_name', 'adj_date'])['revenue_ap2'].sum()

    # Initialize empty lists and DataFrame
    list_po = sorted(data['po_name'].unique())
    list_cols = ['Jumlah Produksi', 'Jumlah Pendapatan', 'Pendapatan PO', 'Pendapatan AP II']
    column_tuples = []
    df_recap = pd.DataFrame()

    # Populate DataFrame with groupby results and calculate sharing
    for po in list_po:
        # Fetch and assign data based on 'po_name'
        df_recap[f'total_produksi_{po}'] = jumlah_produksi.loc[po]
        df_recap[f'total_pendapatan_{po}'] = jumlah_pendapatan.loc[po]
        df_recap[f'pendapatan_{po}'] = pendapatan_po.loc[po]
        df_recap[f'pendapatan_ap2_{po}'] = pendapatan_ap2.loc[po]
        
        # Calculate sharing for the given 'po_name'
        sharing_value = int(data[data['po_name'] == po]['sharing_po'].values[0])
        sharing_factor = np.round(1 - (sharing_value / 100), 2)

        # Prepare column names for MultiIndex
        for col in list_cols:
            column_tuples.append((po, sharing_factor, col))

    # Set MultiIndex column names
    multi_index_columns = pd.MultiIndex.from_tuples(column_tuples)
    df_recap.columns = multi_index_columns

    # Set index format and rename index
    df_recap.index = pd.to_datetime(df_recap.index, format='%Y-%m-%d')
    df_recap.rename_axis('Tanggal Transaksi', inplace=True)

    ## RECAP TOTAL
    print("status: starting recap total")
    # Aggregate all required columns in one go
    aggregated_data = data.groupby('adj_date').agg({
        'price': 'sum',
        'revenue_po': 'sum',
        'revenue_ap2': 'sum',
        'revenue_apsd': 'sum'
    }).reset_index()

    # Rename aggregated columns
    aggregated_data.columns = ['Tanggal Transaksi', 'Total Pendapatan', 'Total Pendapatan PO', 'Total Pendapatan AP II', 'Total Pendapatan APSD']

    # Add 'Total Produksi' column by counting grouped items
    aggregated_data.insert(loc=0, column='Total Produksi', value=data.groupby('adj_date').size().values)
    # aggregated_data['Total Produksi'] = data.groupby('adj_date').size().values

    # Set 'Tanggal Transaksi' as index
    df_total = aggregated_data.set_index('Tanggal Transaksi')

    # Specify the output file and the writer
    print(f"status: exporting file {file_name}.xlsx")
    output_file = f"{file_name}.xlsx"
    with pd.ExcelWriter(output_file) as writer:
        df_recap.to_excel(writer, sheet_name='Rekap per PO', index=True)
        df_total.to_excel(writer, sheet_name='Rekap Total', index=True)
        
    print(f"status: finish")

    


In [5]:
data = get_recap_data(start = '2023-04-01', end = '2023-10-31', file_name = 'Rekap_Dana_BMS-April_Oktober-2023')

In [7]:
# data = collect_data(start = '2023-04-01', end = '2023-10-31')
# data.to_csv('Data_BMS-April_Oktober-2023.csv')